In [1]:
from pymongo import MongoClient, errors
import keepa
import json
import requests
from tqdm import tqdm
import pandas as pd

Initializations

In [2]:
key = '<keepa_key>'
api = keepa.Keepa(key)

domain = 'US'
domain_id = 1

client = MongoClient('mongodb://<username>:<password>@<ip_address>/')

Create and Initialize DB

In [3]:
db = client[f'keepa_db_{domain.lower()}']
categories_collection = db['categories']
best_sellers_collection = db['best_seller_asins']
product_object = db['product_object']
seller_object = db['seller_object']


Functions

In [4]:
def get_tokens_left():    
    base = "https://api.keepa.com/token?key={k}"
    finder = base.format(k=key)
    request_fail = True
    while request_fail:
        try:
            tokens_left = requests.get(finder).json()
            request_fail = False
        except Exception as e:
            continue
    return tokens_left


Getting the Seller data 

In [6]:
category_start_num_sellers = 0
failed_sellers = 0

In [ ]:
seller_list_all = list(product_object.aggregate([{"$unwind": "$offers"}, {"$group":{"_id":'$offers.sellerId'}}]))
print('got product list')
seller_list_all = [x['_id'] for x in seller_list_all]
seller_list_all = list(set(seller_list_all))
print('getting unploaded seller list')
sellers_in_db = list(seller_object.aggregate([{"$group": {"_id": "$_id"}}]))
print('finished getting unploaded seller list')
sellers_in_db = [item['_id'] for item in sellers_in_db]
seller_list = list(set(seller_list_all).difference(set(sellers_in_db)))

seller_list = list(set(seller_list).difference(set([''])))

print('processing sellers')
seller_list = [seller for seller in seller_list if seller is not None]


if len(seller_list) <= 100:
    seller_list_chunks = [seller_list]
else:
    seller_list_chunks = [seller_list[i:i + 100] for i in range(0,len(seller_list),100)]


for sellers in tqdm(seller_list_chunks):
    
    # Getting Product Data
    request_fail = True
    # retry = 0
    while request_fail:
        try:
            seller_query_response = api.seller_query(sellers,domain=domain, wait=True)
            request_fail = False
        except Exception as e:
            # if retry<50:
            #     retry = retry + 1
            # else:
            #     raise
            continue
    
    
    seller_query_response = list(seller_query_response.values())
    for seller_item in seller_query_response:
        seller_item.update( {"_id":seller_item['sellerId']})    
    try:
        seller_object.insert_many(seller_query_response,ordered=False, bypass_document_validation=True)
    except errors.BulkWriteError as e:
        pass
    except UnicodeEncodeError as e:
        for item in seller_query_response:
            try:
                seller_object.insert_one(item, bypass_document_validation=True)
            except Exception as e:
                failed_sellers = failed_sellers+1
                continue
            
category_start_num_sellers = category_start_num_sellers + 1 